## Análise de homologias por BLAST

1. [local_blastp(tag_to_files, db)](#local)
2. [docker_blastp(directory, db)](#docker)
3. [expasy_blastp(tag_to_files)](#expasy)
4. [blastp(tags_and_proteins, db, type)](#blast)

O objectivo desta fase era, dada as proteínas que retirámos do ficheiro genbank, procurar sequências homólogas com BLAST. Esta fase foi iterada várias vezes à procura da melhor estratégia, e no final acabámos com várias alternativas para a execução do BLAST, que podem ser encontradas no módulo [`util.blast`](https://github.com/vitorenesduarte/bionella/blob/master/util/blast.py).

- __Local__
- __Docker__
- __ExPASy__

<a name="local"></a>
### Local

```python
def local_blastp(tag_to_files, db):
    """
    Corre o blast localmente.
    """

    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
    
        blastp_cline = NcbiblastpCommandline(
            query=in_file,
            db=db,
            evalue=10,
            outfmt=5,
            out=out_file
        )
        blastp_cline()
```

<a name="docker"></a>
### Docker

```python
def docker_blastp(directory, db):
    """
    Corre o blast numa instância docker.
    """
    cmd = "docker run -e QUERY_DIR=" + directory \
                  + " -e DB=" + db \
                  + " -v $PWD/" + directory + ":/" + directory \
                  + " -ti vitorenesduarte/swissprot_blast"

    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    p.wait()
```

<a name="expasy"></a>
### ExPASy

```python
def expasy_blastp(tag_to_files):
    """
    Corre o blast na expasy.
    """
    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
        query = rw.read_file(in_file)
        blast_result = www.expasy_blast(query)
        rw.write_file(blast_result, out_file)
```


<a name="blast"></a>
### BLAST

```python
def blastp(tags_and_proteins, db, type="local"):
    """
    Corre o blast para a proteínas passadas como argumento,
    contra a base de dados também passada como argumento.

    O argumento type é opcional e pode ter dois valores:
        - local
        - docker
        - expasy

    É retornado um dicionário com os resultados.
    """

    directory = ".query_dir"
    tag_to_files = write_queries_to_dir(tags_and_proteins, directory)

    # correr o blast
    if type == "local":
        local_blastp(tag_to_files, db)
    elif type == "docker":
        docker_blastp(directory, db)
    elif type == "expasy":
        expasy_blastp(tag_to_files)
    else:
        raise Exception("Unsupported type: " + type)

    blast_results = extract_blast_info(tag_to_files, type)
    return blast_results
```